== This week's links ==

* [The lesson video](https://youtu.be/V2h3IOBDvrA)
* [Lesson 4 notes](http://wiki.fast.ai/index.php/Lesson_4_Notes)


In [1]:
%matplotlib inline
import utils
import imp
imp.reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
path = 'data/ml-latest-small/'
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

## Setup data

We're working with the movielens data, which contains one rating per row, like this:

In [3]:
ratings = pd.read_csv(path + 'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
len(ratings)

100004

Just for displat purposes, let's read in the movie names too.

In [5]:
movie_names = pd.read_csv(path + 'movies.csv').set_index('movieId')['title'].to_dict()

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
userid2idx = {o:i for i, o in enumerate(users)}
movieid2idx = {o:i for i, o in enumerate(movies)}

We update the move and use ids so that they are contiguous integers, which we want when using embeddings.

In [8]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [9]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [10]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

This is the number of latent facors in each embedding.

In [11]:
n_factors = 50
np.random.seed = 42

Randomly split into training and validation.

In [12]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Create subset for Excel

We create a crostab of the most popular movies and most movie-addicted users which we'll copy into excel for creating a simple example. This is not necessary for any of the modeling below however.

In [13]:
g = ratings.groupby('userId')['rating'].count()
topUsers = g.sort_values(ascending = False)[:15]

In [14]:
g = ratings.groupby('movieId')['rating'].count()
topMovies = g.sort_values(ascending = False)[:15]

In [15]:
top_r = ratings.join(topUsers, rsuffix = '_r', how = 'inner', on = 'userId')

In [16]:
top_r = top_r.join(topMovies, rsuffix = '_r', how = 'inner', on = 'movieId')

In [17]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc = np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product

The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:

In [18]:
user_in = Input(shape = (1,), dtype = 'int64', name = 'user_in')
u = Embedding(n_users, n_factors, input_length = 1, embeddings_regularizer = l2(1e-4))(user_in)
movie_in = Input(shape = (1,), dtype = 'int64', name = 'movie_in')
m = Embedding(n_movies, n_factors, input_length = 1, embeddings_regularizer = l2(1e-4))(movie_in)

In [19]:
x = merge([u, m], mode = 'dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss = 'mse')

D:\Users\j00ng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
D:\Users\j00ng\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [20]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 1, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/1
79985/79985 [==============================] - 39s - loss: 9.8195 - val_loss: 4.2756


In [21]:
model.optimizer.lr = 0.01

In [22]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 3, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/3
79985/79985 [==============================] - 12s - loss: 3.1266 - val_loss: 2.8320
Epoch 2/3
79985/79985 [==============================] - 12s - loss: 2.4143 - val_loss: 2.6302
Epoch 3/3
79985/79985 [==============================] - 12s - loss: 2.2485 - val_loss: 2.5807


In [23]:
model.optimizer.lr = 0.001

In [24]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 6, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/6
79985/79985 [==============================] - 11s - loss: 2.1851 - val_loss: 2.5660
Epoch 2/6
79985/79985 [==============================] - 12s - loss: 2.1490 - val_loss: 2.5652
Epoch 3/6
79985/79985 [==============================] - 11s - loss: 2.1228 - val_loss: 2.5619
Epoch 4/6
79985/79985 [==============================] - 11s - loss: 2.0995 - val_loss: 2.5695
Epoch 5/6
79985/79985 [==============================] - 12s - loss: 2.0766 - val_loss: 2.5716
Epoch 6/6
79985/79985 [==============================] - 12s - loss: 2.0571 - val_loss: 2.5816


The [best_benchmarks](http://www.librec.net/example.html) are a bit ober 0.9, so this model doesn't seem to be working that well...

## Bias

The problem is likely yo be taht we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negatibe each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and addin git to our output.

In [25]:
def embedding_input(name, n_in, n_out, reg) :
    inp = Input(shape = (1,), dtype = 'int64', name = name)
    return inp, Embedding(n_in, n_out, input_length = 1, embeddings_regularizer = l2(reg))(inp)

In [26]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [27]:
def create_bias(inp, n_in) :
    x = Embedding(n_in, 1, input_length = 1)(inp)
    return Flatten()(x)

In [28]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [29]:
x = merge([u, m], mode = 'dot')
x = Flatten()(x)
x = merge([x, ub], mode = 'sum')
x = merge([x, mb], mode = 'sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss = 'mse')

D:\Users\j00ng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
D:\Users\j00ng\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
D:\Users\j00ng\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Users\j00ng\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layer

In [30]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 1, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/1
79985/79985 [==============================] - 14s - loss: 8.8058 - val_loss: 3.5344


In [31]:
model.optimizer.lr = 0.01

In [32]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 6, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/6
79985/79985 [==============================] - 13s - loss: 2.5821 - val_loss: 2.3113
Epoch 2/6
79985/79985 [==============================] - 13s - loss: 1.9987 - val_loss: 2.1140
Epoch 3/6
79985/79985 [==============================] - 13s - loss: 1.8388 - val_loss: 2.0168
Epoch 4/6
79985/79985 [==============================] - 13s - loss: 1.7447 - val_loss: 1.9410
Epoch 5/6
79985/79985 [==============================] - 13s - loss: 1.6649 - val_loss: 1.8763
Epoch 6/6
79985/79985 [==============================] - 13s - loss: 1.5902 - val_loss: 1.8055


In [33]:
model.optimizer.lr = 0.001

In [34]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 10, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/10
79985/79985 [==============================] - 14s - loss: 1.5177 - val_loss: 1.7415
Epoch 2/10
79985/79985 [==============================] - 13s - loss: 1.4466 - val_loss: 1.6876
Epoch 3/10
79985/79985 [==============================] - 14s - loss: 1.3794 - val_loss: 1.6274
Epoch 4/10
79985/79985 [==============================] - 13s - loss: 1.3122 - val_loss: 1.5722
Epoch 5/10
79985/79985 [==============================] - 13s - loss: 1.2490 - val_loss: 1.5229
Epoch 6/10
79985/79985 [==============================] - 13s - loss: 1.1887 - val_loss: 1.4747
Epoch 7/10
79985/79985 [==============================] - 14s - loss: 1.1309 - val_loss: 1.4249
Epoch 8/10
79985/79985 [==============================] - 13s - loss: 1.0766 - val_loss: 1.3839
Epoch 9/10
79985/79985 [==============================] - 13s - loss: 1.0247 - val_loss: 1.3423
Epoch 10/10
79985/79985 [==============================] - 14s - loss: 0.9760 - val_lo

In [35]:
model.optimizer.lr = 0.001

In [36]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size = batch_size, epochs = 5, validation_data = ([val.userId, val.movieId], val.rating))

Train on 79985 samples, validate on 20019 samples
Epoch 1/5
79985/79985 [==============================] - 14s - loss: 0.9291 - val_loss: 1.2698
Epoch 2/5
79985/79985 [==============================] - 13s - loss: 0.8861 - val_loss: 1.2379
Epoch 3/5
79985/79985 [==============================] - 13s - loss: 0.8456 - val_loss: 1.2109
Epoch 4/5
79985/79985 [==============================] - 13s - loss: 0.8076 - val_loss: 1.1812
Epoch 5/5
79985/79985 [==============================] - 14s - loss: 0.7730 - val_loss: 1.1553


This result is quite a bit better than the best benchmarks that we could find with a quick google search - so klooks like a greate approach!

In [37]:
model.save_weights(model_path + 'bias.h5')

In [38]:
model.load_weights(model_path + 'bias.h5')

We can use the model to generate predictions by passing a apir of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [39]:
model.predict([np.array([3]), np.array([6])])

array([[ 5.1317]], dtype=float32)

## Analyze results

To make the analysis of the factors more interesting, we'll restrict it to the top 2000 most popular movies.

In [40]:
g = ratings.groupby('movieId')['rating'].count()
topMovies = g.sort_values(ascending = False)[:2000]
topMovies = np.array(topMovies.index)

First, we'll look at the movie bias term. We create a 'model' - which in keras is simply a way of assocoating one or more inputs with one more more outputs, using the functional API. Here, out input is the movie id (a dingle id), and the output is the movie bias (a single float).

In [42]:
get_movie_bias = Model(movie_in, mb)
movie_bias = get_movie_bias.predict(topMovies)
movie_ratings = [(b[0], movie_names[movies[i]]) for i, b in zip(topMovies, movie_bias)]

Now we can look at the top and bottom rated movies. These ratings are corrected for different levels of reviewer sentiment, as well as different types of movies that differenr reviewers watch.

In [43]:
sorted(movie_ratings, key = itemgetter(0))[:15]

[(-0.32078016, 'Battlefield Earth (2000)'),
 (-0.072545812, 'Little Nicky (2000)'),
 (-0.068493322, 'Speed 2: Cruise Control (1997)'),
 (-0.06350217, '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)'),
 (-0.0099264076, 'Road to Wellville, The (1994)'),
 (-0.0080310199, 'Super Mario Bros. (1993)'),
 (-0.0041716117, 'Jaws 3-D (1983)'),
 (0.0073614144, 'Spice World (1997)'),
 (0.0078334454, 'Blade: Trinity (2004)'),
 (0.023951408, 'Howard the Duck (1986)'),
 (0.024372332, 'Wild Wild West (1999)'),
 (0.036260124, 'King Kong (1976)'),
 (0.062546648, 'Police Academy 5: Assignment: Miami Beach (1988)'),
 (0.081072584, 'Police Academy 6: City Under Siege (1989)'),
 (0.084115379, 'Blame It on Rio (1984)')]

In [44]:
sorted(movie_ratings, key = itemgetter(0), reverse = True)[:15]

[(1.420244, 'Band of Brothers (2001)'),
 (1.3878088, 'Blood Simple (1984)'),
 (1.3243535, 'Rush (2013)'),
 (1.305693, 'Porco Rosso (Crimson Pig) (Kurenai no buta) (1992)'),
 (1.3035733, 'Shawshank Redemption, The (1994)'),
 (1.3007047, 'Argo (2012)'),
 (1.298372, "Howl's Moving Castle (Hauru no ugoku shiro) (2004)"),
 (1.2860919, 'Harvey (1950)'),
 (1.2816033, 'General, The (1926)'),
 (1.2768881, 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)'),
 (1.2681067, 'My Neighbor Totoro (Tonari no Totoro) (1988)'),
 (1.2670023, 'Cyrano de Bergerac (1990)'),
 (1.2609829, 'Letters from Iwo Jima (2006)'),
 (1.258718, 'Shall We Dance? (Shall We Dansu?) (1996)'),
 (1.2583145, 'October Sky (1999)')]

We can now so the same thing for the embeddings.

In [45]:
get_movie_emb = Model(movie_in, m)

In [46]:
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

(2000, 50)

Because it's hard to interpret 50 embeddings, we use PCA to simplify them down to just 3 vectors.

In [47]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
movie_pca = pca.fit(movie_emb.T).components_

In [48]:
fac0 = movie_pca[0]

In [51]:
movie_comp = [(f, movie_names[movies[i]]) for f, i in zip(fac0, topMovies)]

Here's the 1st component. It seems to be 'critical acclaimed' or 'classic'.

In [52]:
sorted(movie_comp, key = itemgetter(0), reverse = True)[:10]

[(0.011123721, 'Anaconda (1997)'),
 (0.0091627501, 'Battlefield Earth (2000)'),
 (0.008604561, 'Police Academy 3: Back in Training (1986)'),
 (0.0080690784, 'Jaws 3-D (1983)'),
 (0.0074063931, 'Police Academy 5: Assignment: Miami Beach (1988)'),
 (0.006734618, 'Children of the Corn (1984)'),
 (0.005730594, 'Grease 2 (1982)'),
 (0.0055949017, 'Bio-Dome (1996)'),
 (0.0055649173, 'Barb Wire (1996)'),
 (0.0053354464, "You Don't Mess with the Zohan (2008)")]

In [53]:
sorted(movie_comp, key = itemgetter(0))[:10]

[(-0.053017974, 'Usual Suspects, The (1995)'),
 (-0.051041443, 'Lord of the Rings: The Two Towers, The (2002)'),
 (-0.050995328, 'Lord of the Rings: The Fellowship of the Ring, The (2001)'),
 (-0.05056506, 'Wallace & Gromit: The Wrong Trousers (1993)'),
 (-0.050497621, 'American Beauty (1999)'),
 (-0.050332915, 'Star Wars: Episode V - The Empire Strikes Back (1980)'),
 (-0.050293155, "Schindler's List (1993)"),
 (-0.049606055, 'Wallace & Gromit: A Close Shave (1995)'),
 (-0.049543887, 'Star Wars: Episode IV - A New Hope (1977)'),
 (-0.049326677,
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)')]

In [54]:
fac1 = movie_pca[1]
movie_comp = [(f, movie_names[movies[i]]) for f, i in zip(fac1, topMovies)]

The 2nd is 'hollywood blockbuster'.

In [55]:
sorted(movie_comp, key = itemgetter(0), reverse = True)[:10]

[(0.060884193, 'Annie Hall (1977)'),
 (0.05903247, 'Bringing Up Baby (1938)'),
 (0.05728038, 'City Lights (1931)'),
 (0.055583637, 'Harold and Maude (1971)'),
 (0.054225773, 'Clockwork Orange, A (1971)'),
 (0.053558163, 'Brokeback Mountain (2005)'),
 (0.05302047, 'Apocalypse Now (1979)'),
 (0.050735533, 'Lost in Translation (2003)'),
 (0.050123524, 'Mulholland Drive (2001)'),
 (0.049981549, 'American Psycho (2000)')]

In [56]:
sorted(movie_comp, key = itemgetter(0))[:10]

[(-0.097348645, 'Independence Day (a.k.a. ID4) (1996)'),
 (-0.087436058, 'Armageddon (1998)'),
 (-0.079591341, 'Star Wars: Episode I - The Phantom Menace (1999)'),
 (-0.074939743, 'Stargate (1994)'),
 (-0.073134139, 'Braveheart (1995)'),
 (-0.071750186, 'Outbreak (1995)'),
 (-0.071091473, 'Speed (1994)'),
 (-0.068504922, 'Rock, The (1996)'),
 (-0.066577472, 'American President, The (1995)'),
 (-0.066421844, 'X-Men (2000)')]

In [57]:
fac2 = movie_pca[2]

In [58]:
movie_comp = [(f, movie_names[movies[i]]) for f, i in zip(fac2, topMovies)]

The 3rd is 'violent vs happy'.

In [59]:
sorted(movie_comp, key = itemgetter(0), reverse = True)[:10]

[(0.060767792, 'Elf (2003)'),
 (0.060281143, 'Double Jeopardy (1999)'),
 (0.059373334, 'Tangled (2010)'),
 (0.055298369, 'Roman Holiday (1953)'),
 (0.054659825, 'Superman II (1980)'),
 (0.054153562, 'Pride & Prejudice (2005)'),
 (0.053971373, 'Miss Congeniality (2000)'),
 (0.053791016, 'Coyote Ugly (2000)'),
 (0.052800503, 'Wedding Planner, The (2001)'),
 (0.052509453, 'Legally Blonde (2001)')]

In [60]:
sorted(movie_comp, key = itemgetter(0))[:10]

[(-0.11616777, 'Silence of the Lambs, The (1991)'),
 (-0.10616992, 'Leaving Las Vegas (1995)'),
 (-0.098548047, 'Dumb & Dumber (Dumb and Dumber) (1994)'),
 (-0.096074678, '2001: A Space Odyssey (1968)'),
 (-0.095752515, 'Dances with Wolves (1990)'),
 (-0.095290326, 'Seven (a.k.a. Se7en) (1995)'),
 (-0.091128193, 'Fargo (1996)'),
 (-0.087473109, 'Taxi Driver (1976)'),
 (-0.078661874, "Schindler's List (1993)"),
 (-0.076897264, 'Braveheart (1995)')]

We can draw a picture to see how varioous movies appear on th emap of these components. This picture shows the 1st and 3rd components.

In [ ]:
import sys
stdout , stderr = sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
std.sydout, sys.